In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

configs = [
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='causal', n_trials=500, n_models=1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='mapie', n_trials=500, n_models=1),
    dict(symbol='XAUUSD', timeframe='H1', direction='both',
         train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
         test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
         search_type='lgmm', n_trials=500, n_models=1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
    #      search_type='clusters', search_subtype='simple',
    #      n_trials=500, n_models=1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='clusters', search_subtype='advanced',
    #      n_trials=500, n_models=1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='markov', search_subtype='simple',
    #      n_trials=500, n_models=1),
    # dict(symbol='XAUUSD', timeframe='H1', direction='both',
    #      train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    #      test_start=datetime(2022,1,1),  test_end=datetime(2023,1,1),
    #      search_type='markov', search_subtype='advanced',
    #      n_trials=500, n_models=1),
]

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"]    = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both lgmm] modelo 0 trial 1/500 ins=0.4817 oos=0.3211 avg= 11.80s mem=466.05MB 
[XAUUSD H1 both lgmm] modelo 0 trial 2/500 ins=0.4817 oos=0.3211 avg=  7.51s mem=483.98MB 
[XAUUSD H1 both lgmm] modelo 0 trial 3/500 ins=0.7786 oos=0.4705 avg=  6.34s mem=501.50MB 
[XAUUSD H1 both lgmm] modelo 0 trial 4/500 ins=0.7786 oos=0.4705 avg=  6.43s mem=535.14MB 
[XAUUSD H1 both lgmm] modelo 0 trial 5/500 ins=0.7786 oos=0.4705 avg=  6.43s mem=867.77MB 
[XAUUSD H1 both lgmm] modelo 0 trial 6/500 ins=0.7786 oos=0.4705 avg=  6.38s mem=879.02MB 
